# Step-by-step quick start guide to `CitationExtractor`

# Running the pipeline from the command-line

### Pre-processing

What does it do?:
- tokenization
- language identification
- sentence splitting (optional, depends on configuration)

### Named entity recognition

In [3]:
%%time
citedloci-pipeline do preproc --config=config/project.ini

Fri, 15 Mar 2019 16:06:05 - citation_extractor.Utils.IO - [INFO] Logger initialised
scripts/run_pipeline.py:81: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(open(args['--config']))
Fri, 15 Mar 2019 16:06:05 - citation_extractor.Utils.IO - [INFO] Current working directory: /home/romanell/Documents/ClassicsCitations/IndexLocorum101/data
Fri, 15 Mar 2019 16:06:05 - citation_extractor.Utils.IO - [INFO] Following documents will be processed: [u'bmcr_2013-01-10.txt', u'capra2015_introduction.txt']
Fri, 15 Mar 2019 16:06:05 - citation_extractor.pipeline - [INFO] Env variable $TREETAGGER_HOME == /home/romanell/tree-tagger/
Fri, 15 Mar 2019 16:06:05 - citation_extractor.pipeline - [INFO] Treetagger for fr successfully initialised
Fri, 15 Mar 2019 16:06:05 - citation_extractor.pipeline - [INFO] Treetagger for en successfully initialised
Fri, 15 Mar 2019 16:06:05 - citation_extractor.pipeline - [INFO] Treetagger for nl succ

CPU times: user 596 ms, sys: 3.96 s, total: 4.55 s
Wall time: 2min 49s


Process all files:

In [4]:
citedloci-pipeline do ner --config=config/project.ini

Fri, 15 Mar 2019 16:09:19 - citation_extractor.Utils.IO - [INFO] Logger initialised
scripts/run_pipeline.py:81: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(open(args['--config']))
Fri, 15 Mar 2019 16:09:19 - citation_extractor.Utils.IO - [INFO] Current working directory: /home/romanell/Documents/ClassicsCitations/IndexLocorum101/data
Fri, 15 Mar 2019 16:09:19 - citation_extractor.pipeline - [INFO] Using CitationExtractor v. 1.7.0
Fri, 15 Mar 2019 16:09:20 - citation_extractor.pipeline - [INFO] Training data: found 1 directories containing 936  sentences and 32447 tokens
Fri, 15 Mar 2019 16:09:20 - citation_extractor.core - [INFO] Initialising SklearnClassifier with model CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_sa

Process one specific document (from input folders):

In [10]:
citedloci-pipeline do ner --config=config/project.ini --doc=capra2015_introduction.txt

Fri, 15 Mar 2019 16:11:39 - citation_extractor.Utils.IO - [INFO] Logger initialised
scripts/run_pipeline.py:81: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(open(args['--config']))
Fri, 15 Mar 2019 16:11:39 - citation_extractor.Utils.IO - [INFO] Current working directory: /home/romanell/Documents/ClassicsCitations/IndexLocorum101/data
Fri, 15 Mar 2019 16:11:39 - citation_extractor.pipeline - [INFO] Using CitationExtractor v. 1.7.0
Fri, 15 Mar 2019 16:11:39 - citation_extractor.pipeline - [INFO] Training data: found 1 directories containing 936  sentences and 32447 tokens
Fri, 15 Mar 2019 16:11:39 - citation_extractor.core - [INFO] Initialising SklearnClassifier with model CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_sa

In [9]:
cat data/json/capra2015_introduction.json | jq ".entities"

{
  "T152": {
    "surface": "3",
    "end_offset": 52770,
    "id": "T152",
    "start_offset": 52769,
    "entity_type": "REFSCOPE"
  },
  "T153": {
    "surface": "Plato",
    "end_offset": 53382,
    "id": "T153",
    "start_offset": 53377,
    "entity_type": "AAUTHOR"
  },
  "T150": {
    "surface": "701a.",
    "end_offset": 52248,
    "id": "T150",
    "start_offset": 52243,
    "entity_type": "REFSCOPE"
  },
  "T151": {
    "surface": "Books",
    "end_offset": 52768,
    "id": "T151",
    "start_offset": 52763,
    "entity_type": "REFAUWORK"
  },
  "T156": {
    "surface": "3.37.",
    "end_offset": 53820,
    "id": "T156",
    "start_offset": 53815,
    "entity_type": "REFSCOPE"
  },
  "T157": {
    "surface": "Plato ’s",
    "end_offset": 53900,
    "id": "T157",
    "start_offset": 53892,
    "entity_type": "AAUTHOR"
  },
  "T154": {
    "surface": "Plato ’s",
    "end_offset": 53705,
    "id": "T154",
    "start_offset": 53697,
    "entity_type": "AAUTHOR"
  },
  "T155": {

    "end_offset": 56712,
    "id": "T176",
    "start_offset": 56706,
    "entity_type": "AAUTHOR"
  },
  "T177": {
    "surface": "(Annas",
    "end_offset": 57272,
    "id": "T177",
    "start_offset": 57266,
    "entity_type": "REFAUWORK"
  },
  "T170": {
    "surface": "61a,",
    "end_offset": 56380,
    "id": "T170",
    "start_offset": 56376,
    "entity_type": "REFSCOPE"
  },
  "T171": {
    "surface": "Phaedrus",
    "end_offset": 56389,
    "id": "T171",
    "start_offset": 56381,
    "entity_type": "REFAUWORK"
  },
  "T172": {
    "surface": "248d",
    "end_offset": 56394,
    "id": "T172",
    "start_offset": 56390,
    "entity_type": "REFSCOPE"
  },
  "T173": {
    "surface": "259d,",
    "end_offset": 56404,
    "id": "T173",
    "start_offset": 56399,
    "entity_type": "REFSCOPE"
  },
  "T178": {
    "surface": "1985:401).",
    "end_offset": 57283,
    "id": "T178",
    "start_offset": 57273,
    "entity_type": "REFSCOPE"
  },
  "T179": {
    "surface": "Platon",
    

In [7]:
cat data/json/bmcr_2013-01-10.json | jq ".entities"

{
  "T8": {
    "surface": "12-13,",
    "end_offset": 3751,
    "id": "T8",
    "start_offset": 3745,
    "entity_type": "REFSCOPE"
  },
  "T9": {
    "surface": "Aristophanes,",
    "end_offset": 4803,
    "id": "T9",
    "start_offset": 4790,
    "entity_type": "AAUTHOR"
  },
  "T6": {
    "surface": "3.3)",
    "end_offset": 1359,
    "id": "T6",
    "start_offset": 1355,
    "entity_type": "REFSCOPE"
  },
  "T7": {
    "surface": "Dike",
    "end_offset": 3744,
    "id": "T7",
    "start_offset": 3740,
    "entity_type": "REFAUWORK"
  },
  "T4": {
    "surface": "1.101.2.",
    "end_offset": 1339,
    "id": "T4",
    "start_offset": 1331,
    "entity_type": "REFSCOPE"
  },
  "T5": {
    "surface": "Xen. Hell.",
    "end_offset": 1354,
    "id": "T5",
    "start_offset": 1344,
    "entity_type": "REFAUWORK"
  },
  "T2": {
    "surface": "Thucydides",
    "end_offset": 715,
    "id": "T2",
    "start_offset": 705,
    "entity_type": "AAUTHOR"
  },
  "T3": {
    "surface": "(Thuc.",
